In [ ]:
#%pip install pandas_market_calendars 
import datetime
import pandas_market_calendars as mcal
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import datetime,date ,timedelta

class Memoaize:
    def __init__(self,f) -> None:
        self.f = f
        self.memo = {}
    
    def __call__(self, *args):
        if args not in self.memo:
            self.memo[args] = self.f(*args)
        return self.memo[args]


''' computes the last valid business date. if today is a last valid business date it will return today'''
@Memoaize
def last_valid_business_date(reference_date:date, calendar_name:str ='XLON'):
    # Create a calendar object for the UK market
    calendar = mcal.get_calendar(calendar_name)  # 'XLON' is the code for the London Stock Exchange calendar

    # Get today's date
    today = pd.Timestamp(reference_date)

    if calendar.valid_days(start_date=today, end_date=today).shape[0] > 0:
            return today.date()
        
    # Start from today and go back one day at a time until a valid non-holiday business date is found
    while True:
        today -= timedelta(days=1)
        if calendar.valid_days(start_date=today, end_date=today).shape[0] > 0:
            return today.date()


@Memoaize
def get_next_coupon_date(maturity_date, coupon_freq:int, current_date=None):
    # Define a dictionary to map coupon frequencies to the number of months between coupons.


    if current_date is None:
        current_date = datetime.date.today()

    months_between_coupons = (12/coupon_freq)
    # Calculate the next coupon date.
    next_coupon_date = maturity_date
    while next_coupon_date >= current_date:
        next_coupon_date -= relativedelta(months=months_between_coupons)  # Subtract months using relativedelta
    
    if next_coupon_date < current_date:
        next_coupon_date += relativedelta(months=months_between_coupons)
        
    
    return next_coupon_date

@Memoaize
def get_coupon_ex_date(coupon_date:datetime, days_offset:int, calendar_name="Bond_Markets_UK"):
       # Calculate the next ex-dividend date as 7 business days before the next coupon date.
    uk_calendar = mcal.get_calendar(calendar_name)
    valid_days = uk_calendar.valid_days(start_date=coupon_date - timedelta(days=days_offset+7), 
                                     end_date=coupon_date - timedelta(days=1))
    print(valid_days)
    ex_date = valid_days[-days_offset]
    return ex_date

# Example usage:
ref_date = date(2023, 10, 1)
last_valid_date = last_valid_business_date(ref_date,'XLON')
print("Last Valid Business Date in the UK:", last_valid_date)

# Example usage:
maturity_date = date(2023, 12, 31)  # Replace with the actual maturity date
coupon_frequency = 2
current_date = date(2023, 9, 15)  # Replace with the current date
next_coupon_date = get_next_coupon_date(maturity_date, coupon_frequency, current_date)
next_coupon_ex_date = get_coupon_ex_date(next_coupon_date,7)
print("Next Coupon Date:", next_coupon_date)
print("Next Coupon Ex Date:", next_coupon_ex_date)


In [ ]:
import numpy_financial as npf
import datetime
import pytz

GILTS_COUPON_EX_DATE_OFFSET=7

def calculate_ytm(coupon, face_value, years_to_maturity, price, coupon_freq):
    try:
        coupon = coupon / 100.0
        periods = years_to_maturity * coupon_freq
        ytm = npf.rate(nper=periods, pmt=coupon * face_value / coupon_freq, pv=-price, fv=face_value) * coupon_freq
        return ytm * 100.0
    except Exception as e:
        return None
    
def calculate_accrued_interest(coupon: float, face_value: float, 
                               years_to_maturity: float, payment_frequency: int, 
                               next_coupon_date: datetime, coupon_ex_date_offset:int):
    coupon = coupon / 100.0
    periods = int(years_to_maturity * payment_frequency)
    days_in_period = int(365 / payment_frequency)
    current_date = datetime.datetime.now()
    next_coupon_ex_date = get_coupon_ex_date(next_coupon_date,coupon_ex_date_offset)
    current_date = current_date.replace(tzinfo=pytz.UTC) 
    next_coupon_date = next_coupon_ex_date.replace(tzinfo=pytz.UTC)
  
    days_since_last_coupon = (current_date-next_coupon_ex_date).days % days_in_period
    coupon_payment = (face_value * coupon) / payment_frequency
    accrued_interest = coupon_payment * days_since_last_coupon / days_in_period
    return accrued_interest

def calculate_dirty_price(clean_price: float, accrued_interest: float):
    dirty_price = clean_price + accrued_interest
    return dirty_price

def calculate_time_to_maturity(maturity: datetime, today:datetime=datetime.date.today()):
    return (maturity - today).days / 365

# Example usage:
coupon = 5.0  # 5% coupon rate
face_value = 100.0
years_to_maturity = 5.0
clean_price = 95.0
payment_frequency = 2  # Semi-annual payments
dividend_ex_date = date(2023, 9, 15)  # Replace with the actual dividend ex-date
accrued_interest = calculate_accrued_interest(coupon, face_value, years_to_maturity, payment_frequency, dividend_ex_date,GILTS_COUPON_EX_DATE_OFFSET)
dirty_price = calculate_dirty_price(clean_price, accrued_interest)
print("Dirty Price:", dirty_price)


In [ ]:
maturity_date = date(2025, 3, 7)  # Replace with the actual maturity date
coupon_frequency = 2
current_date = datetime.date(2023, 9, 15)  # Replace with the current date
next_coupon_date = get_next_coupon_date(maturity_date, coupon_frequency, current_date)
next_coupon_ex_date = get_coupon_ex_date(next_coupon_date,GILTS_COUPON_EX_DATE_OFFSET)
print("Next Coupon Date:", next_coupon_date)
print("Next Coupon Ex Date:", next_coupon_ex_date)
assert next_coupon_date ==  date(2024,3,7)
assert next_coupon_ex_date == date(2024,2,27)

print ('another example for calculating next coupon date')
maturity_date = date(2026, 7, 8)  # Replace with the actual maturity date
coupon_frequency = 1
current_date = date(2023, 10, 1)  # Replace with the current date
next_coupon_date = get_next_coupon_date(maturity_date, coupon_frequency, current_date)
print('next coupon date ' + str(next_coupon_date))
assert next_coupon_date == date(2024,7,8)

In [ ]:
bond_source_url = 'https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc'


In [ ]:


# Set the URL to extract the data from
url = bond_source_url
# Use pandas to extract the tables from the URL
tables = pd.read_html(url)
# Select the first table, which contains the bond data
bond_data = tables[0]
# Print the first 10 rows of the bond data
print(bond_data.head(10))


In [ ]:
bond_data.columns


import pandas as pd

url = "https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc"

bond_data = pd.read_html(url)[0]
gilt_ex_date_lag =7 # number of business days prior to the ex-date. 

bond_data['ShortName'] = bond_data['Issuer'].apply(lambda x: x.split('|')[1].strip())
bond_data["Maturity"] = pd.to_datetime(bond_data["Maturity"], format="%d %B %Y")
bond_data['Ttm'] = bond_data["Maturity"].apply(lambda maturity: calculate_time_to_maturity(maturity))
bond_data['CouponFreq']=2
bond_data['NextCouponDate'] = bond_data.apply(lambda row: get_next_coupon_date(row['Maturity'],row['CouponFreq']), axis=1)
bond_data['NextExCouponDate'] = bond_data.apply(lambda row: get_coupon_ex_date(row['NextCouponDate'],gilt_ex_date_lag), 
                                                axis=1)

bond_data=bond_data.rename(columns={'Coupon (%)': 'Coupon'})
bond_data=bond_data.drop(columns=['Issuer','Actions'])
col = bond_data.pop('ShortName')
bond_data.insert(0, 'ShortName', col)

print(bond_data.head())


In [ ]:
bond_data.head

In [ ]:
import pandas as pd

url = "https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc"

bond_data = pd.read_html(url)[0]
gilt_ex_date_lag =7 # number of business days prior to the ex-date. 
date_format ="%d %B %Y"



In [ ]:
bond_data['ShortName'] = bond_data.Issuer.apply(lambda issuer: issuer.split('|')[0].strip());
bond_data['ISIN'] = bond_data.Issuer.apply(lambda issuer: issuer.split('|')[1].strip());

In [ ]:

bond_data['Maturity'] = bond_data.Maturity.apply(lambda maturity: datetime.datetime.strptime(maturity, date_format).date())
print(bond_data.Maturity[-1:])
bond_data['Ttm'] = bond_data.Maturity.apply(calculate_time_to_maturity)
coupon_freq_gilts = 2
bond_data['CouponFreq'] = coupon_freq_gilts
bond_data['NextCouponDate'] = bond_data.Maturity.apply(lambda maturity: get_next_coupon_date(maturity, coupon_freq_gilts))
bond_data['NextCouponExDate'] = bond_data.NextCouponDate.apply(lambda nextCouponDate: get_coupon_ex_date(nextCouponDate, gilt_ex_date_lag))

bond_data=bond_data.rename(columns={'Coupon (%)': 'Coupon'})
bond_data=bond_data.drop(columns=['Actions'])
cols = ['Maturity','NextCouponDate']
for col in cols:
    bond_data[col]= bond_data[col].apply(lambda date: pd.to_datetime(date).date())


In [ ]:
bond_data.head(20)

In [ ]:
bond_data.columns


In [ ]:
FACE_VALUE=100.0

In [ ]:
bond_data.head(1)

In [ ]:
bond_data['AccruedInterest']=bond_data.apply(lambda x: calculate_accrued_interest(x['Coupon'],FACE_VALUE,x['Ttm'],x['CouponFreq'], x['NextCouponDate'], GILTS_COUPON_EX_DATE_OFFSET),axis=1)
bond_data['DirtyPrice']=bond_data.apply(lambda x: calculate_dirty_price(x['Price'],x['AccruedInterest']),axis=1)
bond_data['DirtyYield'] = bond_data.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['DirtyPrice'],x['CouponFreq']), axis=1)
bond_data['CleanYield'] = bond_data.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['Price'],x['CouponFreq']), axis=1)


In [ ]:
bond_data.head()


In [ ]:
bond_data=bond_data.sort_values('Ttm')


In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

bond_data_styler=bond_data.style.set_properties(**{'font-size': '8pt'})
bond_data.sort_values(by='DirtyYield',ascending=False, inplace=True)

In [ ]:
bond_data.head()

In [ ]:
#bond_data.to_excel(r"c:\temp\bond_data.xlsx", index=False)

In [ ]:
# yield filtering 
df_filter_by_maturity = bond_data[bond_data['Ttm']<5]

In [ ]:
df_filter_by_maturity=df_filter_by_maturity.sort_values("DirtyYield", ascending=False)
cols = ['ShortName', 'Coupon', 'Maturity', 'Price', 'Ttm','DirtyPrice', 'DirtyYield', 'CleanYield', 'ISIN', 'NextCouponExDate']
df_less_than_Ttm = df_filter_by_maturity[cols]
df_less_than_Ttm
